In [ ]:
#Failure analysis for Clinicl

import os
import pandas as pd
import numpy as np

dir_path = r'D:\Daneshgah\Tez\ICML2025\data\prescence'
failure_percents = []
#read each csv in directory
for filename in os.listdir(dir_path):
    file_path = os.path.join(dir_path, filename)
    df = pd.read_csv(file_path)
    if filename.endswith('.csv') and filename.startswith('left'):
        df['failure_percent'] = (df['total_frames']-df['frames_with_left_hand'] + df['frames_with_multiple_left_hands'])/df['total_frames']
    else:
        df['failure_percent'] = (df['total_frames']-df['frames_with_right_hand'] + df['frames_with_multiple_right_hands'])/df['total_frames']
        
    failure_percents.append(df['failure_percent'])
    
#calculate the mean and std of failure_percent
failure_percents = pd.concat(failure_percents)
mean = failure_percents.mean()
std = failure_percents.std()
print(f"Mean: {mean}, Std: {std}")
        
    

In [15]:
# Event Counting reading
import os
import pandas as pd
import numpy as np

nt_path = r'D:\Daneshgah\Tez\ICML2025\data\number_of_taps.csv'
df_number_of_taps = pd.read_csv(nt_path)



event_path = r'D:\Daneshgah\Tez\ICML2025\data\hand_event_analysis.csv'
df_event = pd.read_csv(event_path)
#seperate left and right hand finger_tapping analysis
df_taps_alg = pd.DataFrame(columns=['ID', 'hand', 'count'])

df_taps_alg['ID'] = df_event['User_ID']
df_taps_alg['hand'] = 'left'
df_taps_alg['count'] = df_event['left_tap_count']

df_right = df_event[['User_ID', 'right_tap_count']].rename(columns={'User_ID': 'ID', 'right_tap_count': 'count'}).copy()
df_right['hand'] = 'right'
#add df_right to df_taps_alg
df_taps_alg = pd.concat([df_taps_alg, df_right], ignore_index=True)


df_taps_alg = df_taps_alg.join(df_number_of_taps.set_index(['ID','hand'])['Expert_Count'], on=['ID','hand'], rsuffix='_nt')

df_taps_alg.dropna(inplace=True)
df_taps_alg['diff'] = np.abs(df_taps_alg['count'] - df_taps_alg['Expert_Count'])/df_taps_alg['Expert_Count']
#set 1 if it was more than 1
df_taps_alg['diff'] = df_taps_alg['diff'].apply(lambda x: 1 if x > 1 else x)

mean = df_taps_alg['diff'].mean()
std = df_taps_alg['diff'].std()
print(f"Mean: {mean}, Std: {std}")


In [ ]:
#create T-SNE plot

origin_folder = r'\\files.ubc.ca\team\PPRC\Camera\Booth_projects\VQAE'
web_df = pd.read_csv(fr'{origin_folder}\Csvs\web_merge.csv')
web_df.drop(columns=['Video Name','Right Hand Taps','Left Hand Taps','Hand Taps','Expert_Count'], inplace=True)
web_df.set_index('file_name', inplace=True)

#turn tuple to multiply of them
web_df['resolution'] = web_df['resolution'].apply(lambda x: x.replace('(','').replace(')',''))
web_df['resolution'] = web_df['resolution'].apply(lambda x: int(x.split(',')[0])*int(x.split(',')[1])/10000)


#read booth df
booth_df = pd.read_csv(fr'{origin_folder}\Csvs\booth_features.csv')
booth_df.set_index('file_name', inplace=True)
booth_df['tap_diff']  = 0
#show tsne 

import pandas as pd
from sklearn.manifold import TSNE
import plotly.express as px

# Drop NaNs and label column
data = web_df.drop(columns=['tap_diff']).dropna()
labels = web_df['tap_diff'].loc[data.index]


# Fit t-SNE
tsne = TSNE(n_components=2, random_state=22, perplexity=30)
tsne_results = tsne.fit_transform(data)

# Create a DataFrame for plotting
tsne_df = pd.DataFrame(tsne_results, columns=['x', 'y'])
tsne_df['label'] = labels.reset_index(drop=True)

# Plot using Plotly
fig = px.scatter(
    tsne_df, x='x', y='y',
    color='label',
    color_continuous_scale='RdYlGn_r',  # green (low) to red (high)
    labels={'label': 'tap_diff'},
    title='t-SNE visualization',
)

# Customize layout
fig.update_traces(marker=dict(size=10))  # bigger dots
fig.update_layout(
    width=700,
    height=700,
    plot_bgcolor='white',
    paper_bgcolor='white',
    coloraxis_colorbar=dict(title='tap_diff')
)

fig.show()



In [51]:
import pandas as pd
import umap
import plotly.express as px

# Drop NaNs and label column
data = web_df.drop(columns=['tap_diff','resolution']).dropna()
labels = web_df['tap_diff'].loc[data.index]

# Fit UMAP
umap_model = umap.UMAP(n_components=2, random_state=42)
umap_results = umap_model.fit_transform(data)

# Create a DataFrame for plotting
umap_df = pd.DataFrame(umap_results, columns=['x', 'y'])
umap_df['label'] = labels.reset_index(drop=True)

# Plot using Plotly
fig = px.scatter(
    umap_df, x='x', y='y',
    color='label',
    color_continuous_scale='RdYlGn_r',  # green (low) to red (high)
    labels={'label': 'tap_diff'},
    title='UMAP visualization',
)

# Customize layout
fig.update_traces(marker=dict(size=10))  # bigger dots
fig.update_layout(
    width=700,
    height=700,
    plot_bgcolor='white',
    paper_bgcolor='white',
    coloraxis_colorbar=dict(title='tap_diff')
)

fig.show()


In [55]:
import pandas as pd
from sklearn.manifold import TSNE
import umap
import plotly.express as px

# Load and preprocess web_df
origin_folder = r'\\files.ubc.ca\team\PPRC\Camera\Booth_projects\VQAE'
web_df = pd.read_csv(fr'{origin_folder}\Csvs\web_merge.csv')
web_df.drop(columns=['Video Name', 'Right Hand Taps', 'Left Hand Taps', 'Hand Taps', 'Expert_Count'], inplace=True)
web_df.set_index('file_name', inplace=True)
web_df['resolution'] = web_df['resolution'].apply(lambda x: x.replace('(', '').replace(')', ''))
web_df['resolution'] = web_df['resolution'].apply(lambda x: int(x.split(',')[0]) * int(x.split(',')[1]) / 10000)

# Load and preprocess booth_df
booth_df = pd.read_csv(fr'{origin_folder}\Csvs\booth_features.csv')
booth_df.set_index('file_name', inplace=True)
booth_df['tap_diff'] = 0

booth_df['resolution'] = booth_df['resolution'].apply(lambda x: x.replace('(', '').replace(')', ''))
booth_df['resolution'] = booth_df['resolution'].apply(lambda x: int(x.split(',')[0]) * int(x.split(',')[1]) / 10000)

# Combine both datasets (only common columns will be used)
combined_df = pd.concat([web_df, booth_df], axis=0, join='inner')
combined_df = combined_df.dropna()

# Create source label
source = ['web'] * len(web_df) + ['booth'] * len(booth_df)
source = pd.Series(source, index=list(web_df.index) + list(booth_df.index))
source = source.loc[combined_df.index]

# Extract features and labels
features = combined_df.drop(columns=['tap_diff'])
labels = combined_df['tap_diff']

# ========== t-SNE ==========
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
tsne_results = tsne.fit_transform(features)

tsne_df = pd.DataFrame(tsne_results, columns=['x', 'y'])
tsne_df['label'] = labels.reset_index(drop=True)
tsne_df['source'] = source.reset_index(drop=True)

fig_tsne = px.scatter(
    tsne_df, x='x', y='y',
    color='label',
    symbol='source',
    symbol_map={'web': 'star', 'booth': 'circle'},
    color_continuous_scale='RdYlGn_r',
    labels={'label': 'tap_diff', 'source': 'Data Source'},
    title='t-SNE: Web vs Booth'
)

fig_tsne.update_traces(marker=dict(size=10))
fig_tsne.update_layout(
    width=700, height=700,
    plot_bgcolor='white',
    paper_bgcolor='white',
    coloraxis_colorbar=dict(title='tap_diff')
)
fig_tsne.show()

# ========== UMAP ==========
umap_model = umap.UMAP(n_components=2, random_state=42)
umap_results = umap_model.fit_transform(features)

umap_df = pd.DataFrame(umap_results, columns=['x', 'y'])
umap_df['label'] = labels.reset_index(drop=True)
umap_df['source'] = source.reset_index(drop=True)

fig_umap = px.scatter(
    umap_df, x='x', y='y',
    color='label',
    symbol='source',
    symbol_map={'web': 'star', 'booth': 'circle'},
    color_continuous_scale='RdYlGn_r',
    labels={'label': 'tap_diff', 'source': 'Data Source'},
    title='UMAP: Web vs Booth'
)

fig_umap.update_traces(marker=dict(size=10))
fig_umap.update_layout(
    width=700, height=700,
    plot_bgcolor='white',
    paper_bgcolor='white',
    coloraxis_colorbar=dict(title='tap_diff')
)
fig_umap.show()
